# Transfer .h5 model to .pb

In [12]:
# https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb#:~:text=The%20process%20of%20freezing%20a,Dropout%20layers)%20or%20inefficient%20operations


In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [2]:
#model_dir = "./Models/MobileNetV2/MobileNetV2-Birds-Species-FineTune2-97.37.h5"
model_dir = "./Models/MobileNetV2/MobileNetV2-Birds-Species-FineTune1-97.14.h5"
save_dir = "./Models/MobileNetV2/pb/MobileNetV2-Birds-Species-FineTune2-97.14"

In [3]:
# load the model

model = load_model(model_dir)

In [4]:
# model.save(save_dir)

In [5]:
tf.__version__

'2.8.1'

In [6]:
frozen_out_path = "./Models/MobileNetV2/frozen_out"

frozen_graph_filename = "frozen_graph"

In [7]:
#convert keras model to concreteFunction

full_model = tf.function(lambda x: model(x))

In [8]:
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [9]:
# get frozen graph def

frozen_func = convert_variables_to_constants_v2(full_model)
# frozen_func.graph.as_graph_def()

2023-05-28 19:43:34.799549: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-05-28 19:43:34.799778: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-28 19:43:34.801097: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-28 19:43:34.810494: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.238ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.



In [10]:
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

------------------------------------------------------------
Frozen model layers: 
x
model/sequential_3/rescaling_3/Cast/x
model/sequential_3/rescaling_3/mul
model/sequential_3/rescaling_3/Cast_1/x
model/sequential_3/rescaling_3/add
model/mobilenetv2_1.00_224/Conv1/Conv2D/ReadVariableOp/resource
model/mobilenetv2_1.00_224/Conv1/Conv2D/ReadVariableOp
model/mobilenetv2_1.00_224/Conv1/Conv2D
model/mobilenetv2_1.00_224/bn_Conv1/ReadVariableOp/resource
model/mobilenetv2_1.00_224/bn_Conv1/ReadVariableOp
model/mobilenetv2_1.00_224/bn_Conv1/ReadVariableOp_1/resource
model/mobilenetv2_1.00_224/bn_Conv1/ReadVariableOp_1
model/mobilenetv2_1.00_224/bn_Conv1/FusedBatchNormV3/ReadVariableOp/resource
model/mobilenetv2_1.00_224/bn_Conv1/FusedBatchNormV3/ReadVariableOp
model/mobilenetv2_1.00_224/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1/resource
model/mobilenetv2_1.00_224/bn_Conv1/FusedBatchNormV3/ReadVariableOp_1
model/mobilenetv2_1.00_224/bn_Conv1/FusedBatchNormV3
model/mobilenetv2_1.00_224/Conv1_re

In [11]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

'./Models/MobileNetV2/frozen_out/frozen_graph.pbtxt'